In [1]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision as tv

transform_train = tv.transforms.Compose([
    tv.transforms.RandomCrop(32, padding=4),
    tv.transforms.RandomHorizontalFlip(),
    tv.transforms.ToTensor(),
    tv.transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

transform_test = tv.transforms.Compose([
    tv.transforms.ToTensor(),
    tv.transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

train_dataset = tv.datasets.CIFAR10(root='./', train=True, download=True, transform=transform_train)
test_dataset = tv.datasets.CIFAR10(root='./', train=False, download=True, transform=transform_test)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=100, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=100, shuffle=False)

100%|██████████| 170498071/170498071 [00:04<00:00, 35151624.21it/s]


Extracting ./cifar-10-python.tar.gz to ./
Files already downloaded and verified


In [4]:
MODELNAME = "cifar10_vgg16.model"
EPOCHS = 50
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

In [12]:
class VGG16(nn.Module):
    def __init__(self):
        super(VGG16, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(256, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )
        self.classifier = nn.Sequential(
            nn.Linear(512 * 1 * 1, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, 10),
        )

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x

model = VGG16().to(DEVICE)

In [13]:
def train():
    optimizer = optim.Adam(model.parameters(), lr=0.0001)
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.1)
    criterion = nn.CrossEntropyLoss()

    for epoch in range(EPOCHS):
        model.train()
        running_loss = 0.0
        for images, labels in train_loader:
            images = images.to(DEVICE)
            labels = labels.to(DEVICE)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        scheduler.step()
        print(f"Epoch {epoch+1}/{EPOCHS}, Loss: {running_loss}")

    torch.save(model.state_dict(), MODELNAME)

In [14]:
def test():
    model.load_state_dict(torch.load(MODELNAME))
    model.eval()
    correct = 0
    total = len(test_loader.dataset)

    with torch.no_grad():
        for images, labels in test_loader:
            images = images.to(DEVICE)
            labels = labels.to(DEVICE)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            correct += (predicted == labels).sum().item()

    print(f'Correct: {correct}')
    print(f'Total: {total}')
    print(f"Accuracy: {100 * correct / total}%")

In [15]:
train()

Epoch 1/50, Loss: 1001.2247166633606
Epoch 2/50, Loss: 858.4091100692749
Epoch 3/50, Loss: 731.9600386619568
Epoch 4/50, Loss: 640.6352415084839
Epoch 5/50, Loss: 567.7969959378242
Epoch 6/50, Loss: 501.6279647350311
Epoch 7/50, Loss: 446.4034964442253
Epoch 8/50, Loss: 413.7774193882942
Epoch 9/50, Loss: 381.73151218891144
Epoch 10/50, Loss: 351.15896144509315
Epoch 11/50, Loss: 327.7224320471287
Epoch 12/50, Loss: 306.2018339931965
Epoch 13/50, Loss: 286.26026752591133
Epoch 14/50, Loss: 270.93661999702454
Epoch 15/50, Loss: 253.32150445878506
Epoch 16/50, Loss: 241.56750513613224
Epoch 17/50, Loss: 227.0031936466694
Epoch 18/50, Loss: 215.1582535803318
Epoch 19/50, Loss: 203.3438258767128
Epoch 20/50, Loss: 192.63084088265896
Epoch 21/50, Loss: 143.47254614531994
Epoch 22/50, Loss: 134.13038853555918
Epoch 23/50, Loss: 128.9290680065751
Epoch 24/50, Loss: 125.47544783353806
Epoch 25/50, Loss: 122.49192802608013
Epoch 26/50, Loss: 117.81385842710733
Epoch 27/50, Loss: 117.59349628537

In [16]:
test()

Correct: 8625
Total: 10000
Accuracy: 86.25%
